In [1]:
from hello import hello, hello3
import requests
import contextlib

HERR = (hello.HelloError, hello3.HelloError)
NLI = (hello.HelloError, hello3.NotLoggedInError)

from time import sleep
def set_mg(sp):
    call(app.setmg, 1, sp)        
        
def call(f, *args):
    
    while True:
        try:
            rv = f(*args)
        except NLI:
            app.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        else:
            return rv

tests = {
    '3': {
        'air': {
            'air': (),
            'n2' : (),
            'o2' : (),
            'co2': (),

        },
        'mag': {
            'air': ("0.100", "0.300"),
            'n2' : ("0.100", "0.300"),
            'o2' : ("0.100", "0.300"),
            'co2': ("0.030", "0.040"),
        },
    },
    '15': {
        'air': {
            'air': (),
            'n2' : (),
            'o2' : (),
            'co2': (),
        },
        'mag': {
            'air': (),
            'n2' : (),
            'o2' : (),
            'co2': (),
        },
    },
    '80': {
        'air': {
            'air': (),
            'n2' : (),
            'o2' : (),
            'co2': (),
        },
        'mag': {
            'air': ("2.00", "6.00"),
            'n2' : ("2.00", "6.00"),
            'o2' : (),
            'co2': (),
        },
    },
}

def all_off():
    app.login()
    app.setag(2, 12)
    app.setpumpa(0, 0)
    app.setpumpb(0, 0)
    app.setpumpc(0, 0)
    app.setpumpsample(0, 0)
    
def connect(ip=None, v=None):
    v = v or input("Version: ").lower()
    ip = ip or input("Enter IP Address: ")
    if str(v)[0] == "3":
        app = hello3.HelloApp(ip)
    else:
        app = hello.HelloApp(ip)
    return app


def run_test(ip, sz=3, version=3, typ='mag'):
    t = Test(ip, sz, version, typ)
    t.run()
    
    
class Test():
    # State machine 
    def __init__(self, ip, sz, version=3, typ="mag"):
        sz = str(sz)
        typ = typ.lower()
        self.size = sz
        self.type = typ
        self.app = connect(ip, version)
        try:
            self.data = data = tests[sz][typ]
        except KeyError:
            raise KeyError("Invalid size or type: %r %r"%(sz, typ))
        errs = []
        for mfc in data:
            if not data[mfc]:
                errs.append("Don't have complete data for %r"%mfc)
        if errs:
            raise ValueError("\n".join(errs))
        self.cfg = self.getconfig()
        
    def call(self, f, *args):
        f = getattr(self.app, f)
        while True:
            try:
                rv = f(*args)
            except NLI:
                self.app.login()
            except requests.exceptions.ConnectTimeout as e:
                print(str(e))
                print("Make sure you're on the right network!")
            else:
                return rv
            
    def getconfig(self):
        return self.call("getconfig")
    
    def setdo(self, mode, n2, o2):
        return self.call("setdo", mode, n2, o2)
    
    def dooff(self):
        return self.setdo(2, 0, 0)
    
    def setph(self, mode, co2):
        return self.call("setph", mode, co2, 0)
    
    def phoff(self):
        return self.setph(2, 0)        
    
    def setmg(self, sp):
        return self.call("setmg", 1, sp)
    
    @contextlib.contextmanager
    def _tmpcfg(self, group, mode, val):
        oldval = self.cfg[group][mode]
        self.call('setconfig', group, mode, val)
        try:
            yield
        finally:
            self.call('setconfig', group, mode, oldval)
        
    def run(self):
        self.test_air()
        self.test_n2()
        self.test_co2()
        self.test_o2()
        
    def verify(self, mfc, flowrate):
        rv = input("Verify %s %sLPM "%(mfc, flowrate))
        if rv.lower() in ('b', 'q', 'break', 'quit', 's', 'stop'):
            raise KeyboardInterrupt("breaking")
        
    def test_air(self):
        self.test("air")
        
    def test_n2(self):
        with self._tmpcfg("DO", "N2_Manual_Max_(%)", 100):
            self.test("n2", 100)
            
    def test_o2(self):
        with self._tmpcfg("DO", "O2_Manual_Max_(%)", 100):
            self.phoff()
            input("Verify Mass Flow Meter is connected to Micro Gas Outlet and recording Air!!!")
            self.test("o2", 0, 100)
            
    def test_co2(self):
        with self._tmpcfg("pH", "CO2_Manual_Max_(%)", 100):
            self.test("co2", 0, 0, 100)
        
    def test(self, mfc, n2=0, o2=0, co2=0):
        if n2 == o2 == 0:
            self.dooff()
        else:
            self.setdo(1, n2, o2)
        if co2 == 0:
            self.phoff()
        else:
            self.setph(1, co2)
            if self.size == "3":
                m = "0.100"
            elif self.size == "15":
                m = "0.300"
            elif self.size == "80":
                m = "2.00"
            else:
                m = "???"
            self.setmg(m)
            input("Flushing CO2 line. Verify Mass Flow Meter is set to CO2 and press enter when ready!")
        for sp in self.data[mfc]:
            self.setmg(sp)
            self.verify(mfc, sp)

In [2]:
from scripts import wlan
#wlan.ensure_wifi('pbstech')

#         ip               sz  version,        typ
run_test('169.254.110.21', 3,       3,       'mag')

Verify air 0.100LPM 
Verify air 0.300LPM 
Verify n2 0.100LPM 
Verify n2 0.300LPM 
Flushing CO2 line. Verify Mass Flow Meter is set to CO2 and press enter when ready!
Verify co2 0.030LPM 
Verify co2 0.040LPM 
Verify Mass Flow Meter is connected to Micro Gas Outlet and recording Air!!!
Verify o2 0.100LPM .1
Verify o2 0.300LPM 
